In [ ]:
import pickle

import numpy as np
import tensorflow as tf
import gym

from tf_reinforcement_testcases import models

In [ ]:
try:
    with open('data/data.pickle', 'rb') as file:
        init_data = pickle.load(file)
except FileNotFoundError:
    init_data = None

In [ ]:
env = gym.make('CartPole-v1')
n_outputs = env.action_space.n
input_shape = env.observation_space.shape

In [ ]:
model = models.get_mlp(input_shape, n_outputs)
model.set_weights(init_data['weights'])

In [ ]:
def predict(obs):
    obs = tf.nest.map_structure(lambda x: tf.expand_dims(x, axis=0), obs)
    Q_values = model(obs)
    return np.argmax(Q_values[0])

In [ ]:
for i_episode in range(1):
    observation = env.reset()
    for t in range(500):
        env.render()
        # print(observation)
        # action = env.action_space.sample()
        action = predict(observation)
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

In [ ]:
env.close()